In [18]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

In [19]:
from sagemaker.image_uris import retrieve

image_uri = retrieve("xgboost", region="us-east-1", version="0.90-2")
image_uri

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

In [20]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "chapter12"
model_data_prefix = f"s3://{s3_bucket}/multi_model_artifacts/"

In [21]:
from sagemaker.multidatamodel import MultiDataModel

multi_model = MultiDataModel(
    name="MultipleModelsTest",
    model_data_prefix=model_data_prefix, 
    image_uri=image_uri,
    role=role)

In [22]:
%store -r pretrained_model_data_a
%store -r pretrained_model_data_b

In [23]:
multi_model.add_model(pretrained_model_data_a)

's3://sagemaker-cookbook-bucket/multi_model_artifacts/chapter12/files/model.xgboost-1.tar.gz'

In [24]:
multi_model.add_model(pretrained_model_data_b)

's3://sagemaker-cookbook-bucket/multi_model_artifacts/chapter12/files/model.xgboost-2.tar.gz'

In [25]:
list(mdm.list_models())

['chapter12/files/model.xgboost-1.tar.gz',
 'chapter12/files/model.xgboost-2.tar.gz']

In [26]:
model_a, model_b = list(mdm.list_models())
print(model_a)
print(model_b)

chapter12/files/model.xgboost-1.tar.gz
chapter12/files/model.xgboost-2.tar.gz


In [27]:
multi_model.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium', 
    endpoint_name="multi-model-endpoint")

Using already existing model: MultipleModelsTest


---------------------!

In [28]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name="multi-model-endpoint")

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [29]:
predictor.predict(data="11,-5", target_model=model_a)

[0.9153075814247131]

In [30]:
predictor.predict(data="-7,-7", target_model=model_a)

[0.039215054363012314]

In [31]:
predictor.predict(data="11,-5", target_model=model_b)

[0.8308258652687073]

In [32]:
predictor.predict(data="-7,-7", target_model=model_b)

[0.12053093314170837]

In [33]:
predictor.delete_endpoint()